# Amazon SageMaker - Bring Your Own Container¶
### 学習用インスタンスでの学習
ここでは TensorFlow を使ったサンプルコードを題材に、Amazon SageMaker で独自コンテナを用いた機械学習モデルの学習方法を順を追って説明します。トレーニングスクリプトは既に SageMaker 向けに書き換えられた形で準備され、その上で、独自の学習用コンテナを活用します。 トレーニングスクリプトを SageMaker 向けに書き換える際は [Bring Your Own Model ハンズオンワークショップ](https://github.com/aws-samples/amazon-sagemaker-examples-jp/tree/master/workshop/lab_bring-your-own-model) をご参考に下さい。

また本ハンズオンで活用するデータは、[Bring Your Own Container のためのデータ準備](https://github.com/tkazusa/data_prep.ipynb)をお使い下さい。

In [ ]:
import keras

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

sess = sagemaker.Session()

In [ ]:
import sys
!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip install smdebug

In [ ]:
!docker build -t sm-tf-nightly-gpu container/.

## 学習用インスタンスを用いた学習
### ECR への Docker イメージの登録
build した Docker イメージを Docker コンテナレジストリである Amazon ECR へ登録することで、SageMaker の学習時に活用できるようになります。

In [ ]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

ecr_repository = 'sm-tf-nightly-gpu'
image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t {ecr_repository} .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print('コンテナは {} へ登録されています。'.format(image_uri))

## 実験の作成

In [ ]:
import boto3
import time
from smexperiments.experiment import Experiment

boto3_sess = boto3.Session()
sm = boto3_sess.client('sagemaker')

tf_experiment = Experiment.create(
    experiment_name=f"tensorflow-byoc-{int(time.time())}", 
    description="tensorflow experiments", 
    sagemaker_boto_client=sm)

print(tf_experiment)

### GPU を使った学習インスタンスでの学習
登録された Docker イメージと、S3 へアップロードされたデータを用いて学習を行います。
ここでは Optimizer を 'Adam' と 'SGD' とで変更した際の実験管理を行っています。

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent


role = get_execution_role()
bucket = '<your bucket name>'
output_path = 's3://{}/output'.format(bucket)
data_path = {'train': 's3://{}/data'.format(bucket)}

metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss: ([0-9\\.]+)'},
                      {'Name': 'accuracy',
                       'Regex': 'accuracy: ([0-9\\.]+)'}]

trial_name_map = {}

optimizers = ['adam', 'sgd']

for i, optimizer in enumerate(optimizers):
    # Trial の作成
    trial_name = f'tf-training-job-{optimizer}-{int(time.time())}'
    print(trial_name)
    
    
    tf_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=tf_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    trial_name_map[optimizer] = trial_name
    
    # Estimator の作成とfit。
    hyperparameters = {'batch_size': 64,'epochs': 1, 'optimizer': optimizer}
    estimator = Estimator(image_name=image_uri,
                          role=role,
                          hyperparameters=hyperparameters,
                          train_instance_count=1,
                          train_instance_type='ml.p2.xlarge',
                          output_path=output_path,
                          metric_definitions=metric_definitions,
                          enable_sagemaker_metrics=True,)
    
    tf_training_job_name = "tf-job-{}".format(int(time.time()))
    
    estimator.fit(data_path,
                 job_name=tf_training_job_name,
                  
                 experiment_config={
                     'TrialName': tf_trial.trial_name,
                     'TrialComponentDisplayName': 'Training',
                 },
                  
                 wait=False)
    
    time.sleep(2)

## 学習結果の可視化

In [ ]:
from sagemaker.session import Session
from sagemaker.analytics import ExperimentAnalytics

sess = boto3.Session()

search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(sess, sm), 
    experiment_name=tf_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.accuracy.max",
    sort_order="Descending",
    metric_names=['accuracy'],
    parameter_names=['optimizer']
)

analytic_table = trial_component_analytics.dataframe()

In [ ]:
analytic_table